In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import ast
import operator
from typing import Callable, Iterator

import pyarrow.parquet as pq
import pyarrow.compute as pc
import overturemaps
from palettable.colorbrewer.sequential import Reds_6
from palettable.colorbrewer.diverging import RdBu_9
from pathlib import Path
import polars as pl
from deltalake import DeltaTable, write_deltalake
import ast

In [2]:
import lonboard
from lonboard import Map, PolygonLayer, SolidPolygonLayer, viz
from lonboard.colormap import apply_continuous_cmap
import geoarrow.rust.core
from geoarrow.rust.io import read_parquet
import bokeh, ipyleaflet

import geopandas as gpd
import leafmap
from matplotlib.colors import LogNorm
# import leafmap.deckgl as leafmap

In [13]:
import polars.selectors as cs
from great_tables import loc, style, GT

# Cleanup

## Wrong encoding 
Because the encoding is not specified, software will expect UTF-8 but it is not.
This is a hurdle for interoperability purposes.

In [ ]:
def convert_files_to_utf8(files):
    for file in files:
        # Open the CSV file with Latin1 encoding
        df = pd.read_csv(file, encoding='latin1')
        
        # Save the DataFrame to a new CSV file with UTF-8 encoding
        new_file = file.replace('.csv', '_utf8.csv')
        df.to_csv(new_file, encoding='utf-8', index=False)

In [ ]:
files_to_convert = ["solar-gc/municip_potentiel-potential.csv", 
                    "solar-gc/municip_kWh.csv", 
                    "solar-gc/municip_MJ.csv"]

In [ ]:
convert_files_to_utf8(files_to_convert)

## Empty columns

`municip_potentiel-potential.csv` has **10** empty columns !

In [ ]:
problem_file = Path('solar-gc/municip_potentiel-potential_utf8.csv')

In [ ]:
# Read the CSV file
df = pd.read_csv(problem_file)

In [ ]:
df.head()

In [ ]:
# Drop columns with all NaN values
df2 = df.dropna(axis=1, how='all')

In [ ]:
df2.head()

In [ ]:
# Write the updated DataFrame to a new CSV file
df2.to_csv(problem_file, index=False)
# Half a MB gone ...

## Exceedingly complex header
- three lines (instead of one); all files : `municip_potentiel-potential.csv`, `municip_kWh.csv`, `municip_MJ.csv`
- bilingual column names

As a result, the first three columns are unnamed because their first row is empty ...

Schemas need to be in English only.  Translations should be managed elsewhere, not in the dataset.  Plus, bilingual column names mean ludicrous/unmanageable column names.  

This is a MAJOR hurdle for interoperability purposes.  See below.

In [ ]:
from IPython.display import Image

In [ ]:
Image(filename='./solar-gc/malformed_header.png')

In [ ]:
from io import StringIO

def concatenate_columns(csv_data):
    # Read the CSV data
    df = pd.read_csv(StringIO(csv_data))

    # Apply a lambda function to concatenate strings in each row
    df['concatenated'] = df.apply(lambda row: ''.join(map(str, row.values)), axis=1)

    return df['concatenated']

In [ ]:
csv_data_MJ = """
Unnamed: 0,Unnamed: 1,Unnamed: 2,Mean daily global insolation (MJ/m2),Mean daily global insolation (MJ/m2).1,Mean daily global insolation (MJ/m2).2,Mean daily global insolation (MJ/m2).3,Mean daily global insolation (MJ/m2).4,Mean daily global insolation (MJ/m2).5
,,,Ensoleillement global quotidien moyen (MJ/m2),Ensoleillement global quotidien moyen (MJ/m2),Ensoleillement global quotidien moyen (MJ/m2),Ensoleillement global quotidien moyen (MJ/m2),Ensoleillement global quotidien moyen (MJ/m2),Ensoleillement global quotidien moyen (MJ/m2)
Province,Municipality,Mois,South-facing with vertical (90 degrees) tilt,South-facing with latitude tilt,South-facing with tilt=latitude+15 degrees,South-facing with tilt=latitude-15 degrees,2-axis tracking,Horizontal (0 degree)
Province,Municipalité,Month,Vertical orienté vers le sud (inclinaison=90 degrés),Orienté vers le sud avec inclinaison=latitude,Orienté vers le sud avec inclinaison=latitude+15 degrés,Orienté vers le sud avec inclinaison=latitude-15 degrés,Suivi du soleil selon deux axes,Horizontal (inclinaison=0 degré)
"""

In [ ]:
csv_data_MPV = """Unnamed: 0,Unnamed: 1,Unnamed: 2,Photovoltaic potential (kWh/kWp),Photovoltaic potential (kWh/kWp).1,Photovoltaic potential (kWh/kWp).2,Photovoltaic potential (kWh/kWp).3,Photovoltaic potential (kWh/kWp).4,Photovoltaic potential (kWh/kWp).5
,,,Potentiel photovoltaïque (kWh/kWp),Potentiel photovoltaïque (kWh/kWp),Potentiel photovoltaïque (kWh/kWp),Potentiel photovoltaïque (kWh/kWp),Potentiel photovoltaïque (kWh/kWp),Potentiel photovoltaïque (kWh/kWp)
Province,Municipality,Mois,South-facing with vertical (90 degrees) tilt,South-facing with latitude tilt,South-facing with tilt=latitude+15 degrees,South-facing with tilt=latitude-15 degrees,2-axis tracking,Horizontal (0 degree)
Province,Municipalité,Month,Vertical orienté vers le sud (inclinaison=90 degrés),Orienté vers le sud avec inclinaison=latitude,Orienté vers le sud avec inclinaison=latitude+15 degrés,Orienté vers le sud avec inclinaison=latitude-15 degrés,Suivi du soleil selon deux axes,Horizontal (inclinaison=0 degré)
"""

In [ ]:
# Read the CSV data
df = pd.read_csv(StringIO(csv_data_MPV), header=None)
# Initialize a list to store the concatenated columns
concatenated_columns = []

In [ ]:
# Loop through each column and concatenate its values
for col in df.columns:
    column_values = df[col].fillna('').astype(str).tolist()
    concatenated_column = ''.join(column_values)
    concatenated_columns.append(concatenated_column)

In [ ]:
# Print the concatenated columns
for i, column in enumerate(concatenated_columns):
    print(f"Column {i}: {column}")

In [ ]:
# Check the number of rows
print("Number of rows:", len(df))

In [ ]:
# Concatenate each column
concatenated_columns = df.apply(lambda x: ''.join(map(str, x.fillna(''))), axis=0)

In [ ]:
# Check the concatenated column
print(df['concatenated'])

In [ ]:
concatenated_column = concatenate_columns(csv_data_MJ)
len(concatenated_column)
concatenated_column

## Replace header row and delete first three rows

In [ ]:
pv_file = Path('solar-gc/municip_potentiel-potential_utf8.csv')
kwh_file = Path('solar-gc/municip_kWh_utf8.csv')
mj_file = Path('solar-gc/municip_MJ_utf8.csv')

In [ ]:
# Read the CSV file
df_pv = pd.read_csv(pv_file)
df_kwh = pd.read_csv(kwh_file)
df_mj = pd.read_csv(mj_file)

In [ ]:
df_kwh.head()

In [ ]:
new_column_names_mj = ['Province','Municipality','Month','MDGI_mj_south_vert_tilt','MDGI_mj_south_lat_tilt','MDGI_mj_south_lat_plus_15_tilt','MDGI_mj_south_lat_minus_15_tilt','MDGI_mj_two_axis_tracking','MDGI_mj_hor']

new_column_names_kwh = ['Province','Municipality','Month','MDGI_kwh_south_vert_tilt','MDGI_kwh_south_lat_tilt','MDGI_kwh_south_lat_plus_15_tilt','MDGI_kwh_south_lat_minus_15_tilt','MDGI_kwh_two_axis_tracking','MDGI_kwh_hor']

new_column_names_pv = ['Province','Municipality','Month','PV_pot_kWh_p_south_vert_tilt','PV_pot_kWh_p_south_lat_tilt','PV_pot_kWh_p_south_lat_plus_15_tilt','PV_pot_kWh_p_south_lat_minus_15_tilt','PV_pot_kWh_p_two_axis_tracking','PV_pot_kWh_p_hor']

In [ ]:
df_pv.columns = new_column_names_pv
df_kwh.columns = new_column_names_kwh
df_mj.columns = new_column_names_mj

In [ ]:
# remove first three rows
df_pv = df_pv[3:]
df_kwh = df_kwh[3:]
df_mj = df_mj[3:]

## More string manipulation
Province and month

In [ ]:
selected_columns = ['Province','Month']
for column in selected_columns:
    unique_values = df_kwh[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

In [ ]:
province_map = {
    'Alberta/Alberta': 'ALTA',
    'British Columbia/Colombie-Britannique': 'BC',
    'Manitoba/Manitoba': 'MAN',
    'New Brunswick/Nouveau-Brunswick': 'NB',
    'Newfoundland and Labrador/Terre-Neuve-et-Labrador': 'NFL',
    'Northwest Territories/Territoires du Nord-Ouest': 'NWT',
    'Nova Scotia/Nouvelle-Écosse': 'NS',
    'Nunavut/Nunavut': 'NU',
    'Ontario/Ontario': 'ON',
    'Prince Edward Island/île du Prince-Édouard': 'PEI',
    'Quebec/Québec': 'QC',
    'Saskatchewan/Saskatchewan': 'SK',
    'Yukon Territory/Yukon': 'YK'
}

In [ ]:
month_map = {
    'January/Janvier': 'Jan',
    'February/Février': 'Feb',
    'March/Mars': 'Mar',
    'April/Avril': 'Apr',
    'May/Mai': 'May',
    'June/Juin': 'Jun',
    'July/Juillet': 'Jul',
    'August/Août': 'Aug',
    'September/Septembre': 'Sep',
    'October/Octobre': 'Oct',
    'November/Novembre': 'Nov',
    'December/Décembre': 'Dec',
    'Annual/Annuel': 'Annual'
}

In [ ]:
dataframes = [df_pv, df_mj, df_kwh]

In [ ]:
df_pv['Province'] = df_pv['Province'].map(province_map)

In [ ]:
def replace_values(df):
    df['Province'] = df['Province'].apply(lambda x: province_map.get(x, x))
    df['Month'] = df['Month'].apply(lambda x: month_map.get(x, x))
    return df

In [ ]:
# Apply the function to each DataFrame in the list
for i, df in enumerate(dataframes):
    dataframes[i] = replace_values(df)

# Print the updated DataFrames
#for df in dataframes:
#    print(df)

## Save clean files to delta lake

# Pandas will save everything as `str`

In [ ]:
column_names = df_kwh.columns.tolist()[3:]

In [ ]:
column_names

In [ ]:
column_types_pv = {
    'Province':'str',
    'Month':'str',
    'PV_pot_kWh_p_south_vert_tilt': 'int',
    'PV_pot_kWh_p_south_lat_tilt': 'int',
    'PV_pot_kWh_p_south_lat_plus_15_tilt': 'int',
    'PV_pot_kWh_p_south_lat_minus_15_tilt': 'int',
    'PV_pot_kWh_p_two_axis_tracking': 'int',
    'PV_pot_kWh_p_hor': 'int',
    # Add more columns and their types as needed
}

In [ ]:
column_types_mj = {
    'Province':'str',
    'Month':'str',
    'MDGI_mj_south_vert_tilt': 'float',
    'MDGI_mj_south_lat_tilt': 'float',
    'MDGI_mj_south_lat_plus_15_tilt': 'float',
    'MDGI_mj_south_lat_minus_15_tilt': 'float',
    'MDGI_mj_two_axis_tracking': 'float',
    'MDGI_mj_hor': 'float',
    # Add more columns and their types as needed
}

In [ ]:
column_types_kwh = {
    'Province':'str',
    'Month':'str',
    'MDGI_kwh_south_vert_tilt': 'float',
    'MDGI_kwh_south_lat_tilt': 'float',
    'MDGI_kwh_south_lat_plus_15_tilt': 'float',
    'MDGI_kwh_south_lat_minus_15_tilt': 'float',
    'MDGI_kwh_two_axis_tracking': 'float',
    'MDGI_kwh_hor': 'float',
    # Add more columns and their types as needed
}

In [ ]:
df_mj = df_mj.astype(column_types_mj) # and other dfs

In [ ]:
write_deltalake('./solar-gc/pv', df_pv)

In [ ]:
write_deltalake('./solar-gc/mj', df_mj)

In [ ]:
write_deltalake('./solar-gc/kwh', df_kwh)

## Unnecessary information

MJ/m² et kWh/m² : No need for two files since it is just a conversion factor !!

1 kWh/m² =  3.597122 mJ/m²

1 MJ/m² =  0.278 kWh/m²

Every 12 rows : Annual/Annuel; 
- not necessary (can be computed)
- is misleading!  should be "Annual mean"

# Work with delta tables

In [4]:
pldf_pv = pl.read_delta("./solar-gc/pv")

In [5]:
pldf_mj = pl.read_delta("./solar-gc/mj")

In [6]:
pldf_kwh = pl.read_delta("./solar-gc/kwh")

In [10]:
pldf_pv

Province,Municipality,Month,PV_pot_kWh_p_south_vert_tilt,PV_pot_kWh_p_south_lat_tilt,PV_pot_kWh_p_south_lat_plus_15_tilt,PV_pot_kWh_p_south_lat_minus_15_tilt,PV_pot_kWh_p_two_axis_tracking,PV_pot_kWh_p_hor
str,str,str,i32,i32,i32,i32,i32,i32
"""ALTA""","""Acadia Valley""","""Jan""",88,83,89,72,100,29
"""ALTA""","""Acadia Valley""","""Feb""",98,100,104,91,123,47
"""ALTA""","""Acadia Valley""","""Mar""",111,130,128,125,167,84
"""ALTA""","""Acadia Valley""","""Apr""",92,130,120,134,190,112
"""ALTA""","""Acadia Valley""","""May""",78,130,114,141,211,136
…,…,…,…,…,…,…,…,…
"""YK""","""Whitehorse""","""Sep""",68,83,78,83,109,59
"""YK""","""Whitehorse""","""Oct""",49,53,53,51,64,28
"""YK""","""Whitehorse""","""Nov""",30,30,31,27,34,10


# Write to FSDH
Does not work yet

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient

In [ ]:
credential = DefaultAzureCredential()

In [ ]:
credential

In [ ]:
blob_service_client = BlobServiceClient(
    account_url="https://fsdhprojteympoc.blob.core.windows.net/",
    credential=credential
)

In [ ]:
blob_service_client

In [ ]:
storage_opts = {
    "anon": False,
    "credential": credential
}

In [ ]:
pldf_pv.write_delta(
    "wasbs://datahub@fsdhprojteympoc.blob.core.windows.net/pv",
    storage_options=storage_opts
)

# Dice and slice

In [ ]:
pldf_pv.style.tab_style(
    style.fill("yellow"),
    loc.body(rows=pl.col("PV_pot_kWh_p_south_vert_tilt") == pl.col("PV_pot_kWh_p_south_vert_tilt").max()),
)  


In [ ]:
pldf_pv.head(10).style.tab_spanner(
    "Photovoltaic potential", cs.starts_with("PV")
)  

In [19]:
# Colors at https://natural-resources.canada.ca/energy/energy-sources-distribution/renewables/solar-photovoltaic-energy/tools-solar-photovoltaic-energy/photovoltaic-and-solar-resource-maps/18366
# Annual PV potential; south facing with latitude tilt
gc_pv_colors = {
    '0-500': '#010080',
    '500-600': '#0000CC',
    '600-700': '#0099FF',
    '700-800': '#009999',
    '800-900': '#01CC00',
    '900-1000': '#00FF01',
    '1000-1100': '#CCFF00',
    '1100-1200': '#FFFF00',
    '1200-1300': '#FFCC00',
    '1300-1400': '#FE9900',
    '1400+': '#FE332D'
}

In [20]:
import IPython.display as ipd

for interval, color in gc_pv_colors.items():
    ipd.display(ipd.HTML(f"<span style='background-color:{color}; color:white; padding:5px'>{interval}</span>"))